## get dataset

In [56]:
# import kagglehub

# Download latest version
# path = kagglehub.dataset_download("adarshsng/lending-club-loan-data-csv")
# print("Path to dataset files:", path)


## data cleaning

In [74]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_DIR = Path("data")
SAMPLE_PATH = DATA_DIR / "loan_sample_100m.csv"

df = pd.read_csv(SAMPLE_PATH, low_memory=False)
print("Shape:", df.shape)
df.head(3)
# 基础信息：类型与关键列
df.dtypes[df.dtypes == "object"].index.tolist()  # 对象型列
df.info()

Shape: (199999, 145)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Columns: 145 entries, id to settlement_term
dtypes: float64(59), int64(51), object(35)
memory usage: 221.3+ MB


In [75]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


### 只保留分析用列（去掉大量为空的列）

按 `docs/dataset.md` 保留转化/风险/收益/用户分群分析会用到的列，其余删除，避免 145 列里大量全空列干扰。

In [58]:
# 分析用列（见 docs/dataset.md）：基础/结果、转化、风险、收益、用户分群
COLS_TO_KEEP = [
    "loan_status", "issue_d", "loan_amnt", "funded_amnt", "funded_amnt_inv",
    "term", "int_rate", "installment", "grade", "sub_grade", "verification_status",
    "purpose", "addr_state", "zip_code", "title",
    "dti", "annual_inc", "emp_length", "emp_title", "home_ownership",
    "delinq_2yrs", "mths_since_last_delinq", "pub_rec", "revol_bal", "revol_util",
    "open_acc", "total_acc", "inq_last_6mths", "inq_last_12m", "earliest_cr_line",
    "collections_12_mths_ex_med", "chargeoff_within_12_mths",
    "total_pymnt", "total_rec_int", "total_rec_prncp", "last_pymnt_d", "last_pymnt_amnt",
    "out_prncp", "out_prncp_inv",
    "avg_cur_bal", "bc_util", "all_util", "tot_cur_bal", "total_rev_hi_lim",
    "acc_now_delinq", "num_accts_ever_120_pd", "num_tl_90g_dpd_24m", "pub_rec_bankruptcies",
    "application_type",
]
existing = [c for c in COLS_TO_KEEP if c in df.columns]
df = df[existing].copy()
print("保留列数:", len(existing), "| 删除列数:", 145 - len(existing))
df.head(10)

保留列数: 49 | 删除列数: 96


,loan_status,issue_d,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,avg_cur_bal,bc_util,all_util,tot_cur_bal,total_rev_hi_lim,acc_now_delinq,num_accts_ever_120_pd,num_tl_90g_dpd_24m,pub_rec_bankruptcies,application_type
0,Current,Dec-2018,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,1878.0,5.9,28.0,16901,42000,0,0,0,1,Individual
1,Current,Dec-2018,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,24763.0,8.3,57.0,321915,50800,0,0,0,1,Individual
2,Current,Dec-2018,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,18383.0,0.0,35.0,110299,24100,0,0,0,0,Individual
3,Current,Dec-2018,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,30505.0,75.2,70.0,305049,7000,0,0,0,0,Individual
4,Current,Dec-2018,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,9667.0,8.9,54.0,116007,23100,0,0,0,0,Individual
5,Current,Dec-2018,5550,5550,5550.0,36 months,15.02,192.45,C,C3,...,40338.0,64.0,58.0,685749,111900,0,0,0,0,Individual
6,Current,Dec-2018,2000,2000,2000.0,36 months,17.97,72.28,D,D1,...,854.0,NaN,100.0,854,0,0,0,0,0,Individual
7,Current,Dec-2018,6000,6000,6000.0,36 months,13.56,203.79,C,C1,...,5085.0,90.8,74.0,91535,55500,0,0,0,0,Individual
8,Current,Dec-2018,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,5235.0,35.9,73.0,41882,22800,0,2,0,0,Individual
9,Current,Dec-2018,6000,6000,6000.0,36 months,14.47,206.44,C,C2,...,9197.0,60.6,48.0,349502,132500,0,0,0,0,Individual


### 缺失值


In [59]:
null_count = df.isnull().sum()
null_pct = (null_count / len(df) * 100).round(2)
missing_df = pd.DataFrame({"n_null": null_count, "pct": null_pct})
# 只保留有缺失的列，按缺失数从多到少排序（供查看）
missing_df = missing_df[missing_df["n_null"] > 0].sort_values("n_null", ascending=False)
missing_df.head(30)


,n_null,pct
mths_since_last_delinq,112455,56.23
emp_title,29627,14.81
emp_length,17852,8.93
bc_util,2704,1.35
dti,392,0.20
last_pymnt_d,241,0.12
revol_util,231,0.12
all_util,56,0.03
avg_cur_bal,21,0.01


In [60]:
# 空值 = 无逾期，填 180 月（15 年）表示“距上次逾期很久/从未逾期”
if "mths_since_last_delinq" in df.columns:
    n_fill = df["mths_since_last_delinq"].isna().sum()
    df["mths_since_last_delinq"] = df["mths_since_last_delinq"].fillna(180)
    print(f"mths_since_last_delinq 补全: {n_fill} 个空值 → 120（无逾期）")
df["mths_since_last_delinq"].describe()

mths_since_last_delinq 补全: 112455 个空值 → 120（无逾期）


count    199999.000000
mean        117.372307
std          72.436955
min           0.000000
25%          39.000000
50%         180.000000
75%         180.000000
max         180.000000
Name: mths_since_last_delinq, dtype: float64

In [61]:

# 去掉存在缺失值的行（列不删）
before = len(df)
df = df.dropna()
print(f"去掉存在缺失值的行: {before} -> {len(df)}，删除 {before - len(df)} 行")

去掉存在缺失值的行: 199999 -> 167981，删除 32018 行


### 好坏客户定义（is_default）

根据 `loan_status` 定义违约：Charged Off、Default、各类 Late 等视为坏（1），Current、Fully Paid 等视为好（0）；其余（如 In Grace Period）可单独处理。

In [62]:
# loan_status 取值
df["loan_status"].value_counts(dropna=False)

loan_status
Current               160003
Fully Paid              6003
Late (31-120 days)       881
In Grace Period          744
Late (16-30 days)        251
Charged Off               99
Name: count, dtype: int64

In [63]:
# 违约状态：Charged Off、Default、Late 等
BAD_STATUS = [
    "Charged Off", "Default", "Does not meet the credit policy. Status:Charged Off",
    "Does not meet the credit policy. Status:Grace Period",
    "Late (31-120 days)", "Late (16-30 days)", "In Grace Period"
]
df["is_default"] = df["loan_status"].isin(BAD_STATUS).astype(int)
# 仅对已结清/已违约的样本算违约率时，可再筛：Fully Paid + 上述坏状态
df["loan_status"].value_counts()
df["is_default"].value_counts()

is_default
0    166006
1      1975
Name: count, dtype: int64

In [64]:
# int_rate：可能是 "13.56" 或 "13.56%"
if df["int_rate"].dtype == object:
    df["int_rate"] = pd.to_numeric(df["int_rate"].astype(str).str.replace("%", "", regex=False), errors="coerce")
# term：提取月数
if "term" in df.columns and df["term"].dtype == object:
    df["term_months"] = pd.to_numeric(df["term"].astype(str).str.extract(r"(\d+)", expand=False), errors="coerce")
# issue_d：转为日期（格式如 Dec-2018）
if "issue_d" in df.columns:
    df["issue_date"] = pd.to_datetime(df["issue_d"], format="%b-%Y", errors="coerce")
df[["int_rate", "term", "term_months", "issue_date"]].head(3)

,int_rate,term,term_months,issue_date
0,13.56,36 months,36,2018-12-01
1,18.94,60 months,60,2018-12-01
2,17.97,36 months,36,2018-12-01


### 清洗小结

- 去重、`is_default` 定义、`int_rate`/`term`/`issue_d` 格式已处理。
- 分析时使用 `df_clean`；需保存时可导出为 CSV。

In [65]:
# 清洗后概览
df.describe(include="all")

,loan_status,issue_d,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,tot_cur_bal,total_rev_hi_lim,acc_now_delinq,num_accts_ever_120_pd,num_tl_90g_dpd_24m,pub_rec_bankruptcies,application_type,is_default,term_months,issue_date
count,167981,167981,167981.000000,167981.000000,167981.000000,167981,167981.000000,167981.000000,167981,167981,...,1.679810e+05,1.679810e+05,167981.0,167981.000000,167981.000000,167981.000000,167981,167981.000000,167981.000000,167981
unique,6,5,NaN,NaN,NaN,2,NaN,NaN,7,35,...,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN
top,Current,Oct-2018,NaN,NaN,NaN,36 months,NaN,NaN,A,A1,...,NaN,NaN,NaN,NaN,NaN,NaN,Individual,NaN,NaN,NaN
freq,160003,38403,NaN,NaN,NaN,113779,NaN,NaN,50701,12339,...,NaN,NaN,NaN,NaN,NaN,NaN,147076,NaN,NaN,NaN
mean,NaN,NaN,16321.180818,16321.180818,16318.340089,NaN,12.856414,471.941622,NaN,NaN,...,1.526429e+05,3.999457e+04,0.0,0.464154,0.059102,0.116531,NaN,0.011757,43.744019,2018-10-03 00:24:06.335002112
min,NaN,NaN,1000.000000,1000.000000,725.000000,NaN,6.000000,30.480000,NaN,NaN,...,0.000000e+00,2.000000e+02,0.0,0.000000,0.000000,0.000000,NaN,0.000000,36.000000,2018-08-01 00:00:00
25%,NaN,NaN,9000.000000,9000.000000,9000.000000,NaN,8.460000,261.270000,NaN,NaN,...,3.030800e+04,1.770000e+04,0.0,0.000000,0.000000,0.000000,NaN,0.000000,36.000000,2018-09-01 00:00:00
50%,NaN,NaN,14500.000000,14500.000000,14500.000000,NaN,11.800000,395.160000,NaN,NaN,...,8.387100e+04,3.060000e+04,0.0,0.000000,0.000000,0.000000,NaN,0.000000,36.000000,2018-10-01 00:00:00
75%,NaN,NaN,22400.000000,22400.000000,22400.000000,NaN,16.140000,634.080000,NaN,NaN,...,2.312480e+05,5.070000e+04,0.0,0.000000,0.000000,0.000000,NaN,0.000000,60.000000,2018-11-01 00:00:00
max,NaN,NaN,40000.000000,40000.000000,40000.000000,NaN,30.990000,1618.240000,NaN,NaN,...,9.971659e+06,1.514001e+06,0.0,58.000000,58.000000,6.000000,NaN,1.000000,60.000000,2018-12-01 00:00:00


In [66]:
# 可选：导出清洗后数据供 SQL / 仪表板使用
df.to_csv(DATA_DIR / "loan_clean.csv", index=False)
print("已保存至 data/loan_clean.csv")

已保存至 data/loan_clean.csv
